In [1]:
def run_PBIL_experiments(number_of_runs, **kwargs):
    costs = None
    probabilities = []
    epochs = kwargs.pop("epochs")
    def run_single_experiment(run):
        pbil_run = PBIL(**kwargs, name=run)
        pbil_run.evolve(epochs=epochs)
        return pbil_run.get_scores(), pbil_run.get_historical_probabilities()
    
    experiments = Parallel(n_jobs=10, verbose=10)(delayed(run_single_experiment)(i) for i in range(number_of_runs))

    costs, probabilities = zip(*experiments)
    fig = make_subplots(rows=1, cols=2)
    for line, values in costs[0].items():
        fig.add_trace(
            go.Scatter(y=values, mode='lines', name=line), row=1, col=1
        )

    for i, history in enumerate([[sum(probability_vector)/kwargs["chromosome_length"] for probability_vector in history]
                      for history in probabilities]):
        fig.add_trace(
            go.Scatter(y=history, mode='lines', name=i), row=1, col=2
        )

    fig.update_layout(title_text="PBIL Experiment")
    fig.show()